In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import numpy as np
import warnings
import re
warnings.filterwarnings('ignore',category=FutureWarning)


html = urlopen('https://storm.cis.fordham.edu/~yli/data/LoveOutput/').read().decode('utf-8','ignore')
soup = BeautifulSoup(html, features='lxml')

all_href = soup.find_all('a')
all_href = [l['href'] for l in all_href]


def lAndP(t):
    p = re.sub(r'<\w+/><\w+/>',' [P]',t)
    l = re.sub(r'<\w+?/>',' [L]',p)
    l = re.sub(r'<.*?>','',l)
    l = re.sub(r'\xa0','',l)
    return l

def null(l):
    if l == []:
        return [np.NAN]
    return l


text = []
n=1
for i in all_href[5:]:

    html = urlopen('https://storm.cis.fordham.edu/~yli/data/LoveOutput/'+i).read()
    soup = BeautifulSoup(html, features='lxml')
    a = str(soup).split('\r\n\r\n')[2]
    t = soup.text
    
    author = re.findall(r'(?<=\sBy\s)(.*?)\r',t)[0]
    title = re.findall(r'.+(?=\sBy)',t)[0]
    tags = null(re.findall(r'\;(\w+[\s|&]\w+|\w+)',t))
    body = lAndP(a)
    link = null(re.findall(r'https:.+',t))[0]
    
    data = np.array([n,author,title,tags,body,link])
    n+=1
    text.append(data)



In [2]:
import pandas as pd
df = pd.DataFrame(text,columns=['ID','Author','Title','Tags','Body','Link'])
df_clean = df.dropna(axis=0)
df_clean.to_excel('CleanOutputLoveOutput.xlsx',index=False)

In [3]:
#c.LengthOne: the total number of tokens in this poem, punctuation marks are not included.

from nltk.tokenize import sent_tokenize
from collections import Counter
def getLength_c(text):
    l = re.sub(r'\[[L|P]\]','',text)
    return len(l.split())

def getLength_d(text):
    return len(text.split())

def getNum_e(text):
    counter = Counter(text.split())
    return counter['[L]'] + counter['[P]']

def getNum_f(text):
    counter = Counter(text.split())
    return counter['[P]']

def getNum_g(text):
    
    return len(sent_tokenize(text))

def getNum_h(text):
    
    return len(text.split(','))-1

    


In [4]:
df_q2 = df_clean[['ID','Author']]

df_q2["LengthOne"] = df_clean["Body"].apply(lambda text: getLength_c(text))
df_q2["LengthTwo"] = df_clean["Body"].apply(lambda text: getLength_d(text))                 
df_q2["NumLine"] = df_clean["Body"].apply(lambda text: getNum_e(text))                 
df_q2["NumPara"] = df_clean["Body"].apply(lambda text: getNum_f(text))                 
df_q2["NumSent"] = df_clean["Body"].apply(lambda text: getNum_g(text))                 
df_q2["NumComma"] = df_clean["Body"].apply(lambda text: getNum_h(text))                 
df_q2.head()               

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

,ID,Author,LengthOne,LengthTwo,NumLine,NumPara,NumSent,NumComma
0,1,Suzanne Gardinier,114,128,14,7,1,0
1,2,Jeff Daniel Marion,85,86,1,1,2,10
2,3,Roddy Lumsden,290,292,2,1,17,19
3,4,Samuel Menashe,29,51,22,1,1,0
4,5,Dennis Cooper,162,201,39,1,25,5


In [5]:
df_1 = df_q2[['ID','Author']]
df_2 = df_q2[['ID','Author']]
df_3 = df_q2[['ID','Author']]


from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

def remove(text):
    return re.sub(r'\[[L|P]\]','',text)


def getSWtoker(text):
    l = remove(text)
    l = remove_stopwords(l)
    return word_tokenize(l)

def getSStoker(text):
    l = remove(text)
    l = remove_stopwords(l)
    l = stem_words(l)
    return word_tokenize(l)

def plainToker(text):
    l = remove(text)
    return word_tokenize(l)




In [11]:
df_1["Body"] = df_clean["Body"].apply(lambda text: plainToker(text))
df_1["Length"] = df_1["Body"].apply(lambda text: len(text))
df_1["UnitCount"] = df_1["Body"].apply(lambda text: len(set(text)))

df_2["Body"] = df_clean["Body"].apply(lambda text: getSWtoker(text))
df_2["Length"] = df_2["Body"].apply(lambda text: len(text))
df_2["UnitCount"] = df_2["Body"].apply(lambda text: len(set(text))) 

df_3["Body"] = df_clean["Body"].apply(lambda text: getSStoker(text))
df_3["Length"] = df_3["Body"].apply(lambda text: len(text))
df_3["UnitCount"] = df_3["Body"].apply(lambda text: len(set(text)))  

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/p

In [12]:
df_1

,ID,Author,Body,Length,UnitCount
0,1,Suzanne Gardinier,"[I, 'm, used, to, the, emperor, 's, bitterness...",120,72
1,2,Jeff Daniel Marion,"[In, the, back, of, the, junkhouse, stacked, o...",100,69
2,3,Roddy Lumsden,"[They, arrived, at, the, desk, of, the, Hotel,...",333,198
3,4,Samuel Menashe,"[A-, round, my, neck, an, amu-, let, Be-, twee...",29,26
4,5,Dennis Cooper,"[for, Brad, Gooch, We, snort, all, our, coke, ...",203,108
5,6,Kathryn Maris,"[Identification, is, a, highly, important, fac...",329,169
6,7,Thomas Moore,"[Written, at, Norfolk, ,, in, Virginia, “, The...",362,188
7,8,Christina Rossetti,"[My, heart, is, like, a, singing, bird, Whose,...",120,69
8,9,Jane Hirshfield,"[Today, when, persimmons, ripen, Today, when, ...",188,102
9,10,Lewis Carroll,"[A, boat, beneath, a, sunny, sky, ,, Lingering...",130,84


In [13]:
df_2

,ID,Author,Body,Length,UnitCount
0,1,Suzanne Gardinier,"[I, 'm, used, emperor, 's, bitterness, I, ca, ...",81,55
1,2,Jeff Daniel Marion,"[In, back, junkhouse, stacked, cardtable, cove...",64,52
2,3,Roddy Lumsden,"[They, arrived, desk, Hotel, Duncanand, Smithe...",202,156
3,4,Samuel Menashe,"[A-, round, neck, amu-, let, Be-, tween, eyes,...",17,17
4,5,Dennis Cooper,"[Brad, Gooch, We, snort, coke, way, party, ., ...",143,83
5,6,Kathryn Maris,"[Identification, highly, important, factor, me...",196,127
6,7,Thomas Moore,"[Written, Norfolk, ,, Virginia, “, They, made,...",265,158
7,8,Christina Rossetti,"[My, heart, like, singing, bird, Whose, nest, ...",78,54
8,9,Jane Hirshfield,"[Today, persimmons, ripen, Today, fox-kits, co...",108,76
9,10,Lewis Carroll,"[A, boat, beneath, sunny, sky, ,, Lingering, o...",98,65


In [ ]:
df_3

,ID,Author,Body,Length,UnitCount
0,1,Suzanne Gardinier,"[i, 'm, use, emperor, ', bitter, I, ca, n't, f...",81,55
1,2,Jeff Daniel Marion,"[In, back, junkhous, stack, cardtabl, cover, r...",62,51
2,3,Roddy Lumsden,"[they, arriv, desk, hotel, duncanand, smith, i...",200,153
3,4,Samuel Menashe,"[A-, round, neck, amu-, let, be-, tween, eye, ...",17,17
4,5,Dennis Cooper,"[brad, gooch, We, snort, coke, way, party, ., ...",142,80
5,6,Kathryn Maris,"[identif, highli, import, factor, mechan, hyst...",194,127
6,7,Thomas Moore,"[written, norfolk, ,, virginia, “, they, made,...",262,155
7,8,Christina Rossetti,"[My, heart, like, sing, bird, whose, nest, wat...",78,54
8,9,Jane Hirshfield,"[today, persimmon, ripen, today, fox-kit, come...",108,73
9,10,Lewis Carroll,"[A, boat, beneath, sunni, sky, ,, linger, onwa...",98,64


In [ ]:
q2excel = pd.ExcelWriter('ProcessedLoveOutput.xlsx')
df_q2.to_excel(q2excel,'Love_ST',index = False)
df_1.to_excel(q2excel,'Love_T',index = False)
df_2.to_excel(q2excel,'Love_S2',index = False)
df_3.to_excel(q2excel,'Love_F',index = False)
q2excel.save()
q2excel.close()